In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from modules.utils.data_processing import get_match_label

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

### Create database connection

In [2]:
database_path = "datasets/database.sqlite"
conn = sqlite3.connect(database_path)

### Read data from database

In [3]:
match_data_sql = '''select * from Match '''
team_attributes_sql = '''select * from Team_Attributes'''
player_attributes_sql = '''select player_api_id, overall_rating from Player_Attributes'''
# Country, League and Team tables don't seem to have any value for us, country_id is equivalent to country_name 
# for the neural net, as well as league_id/league name, team_id/team name
# TODO Player table may be beneficial due to some features (birthyear, weight, height), might want to add them later

match_data = pd.read_sql(match_data_sql, conn)
team_attributes = pd.read_sql(team_attributes_sql, conn)
player_attributes = pd.read_sql(player_attributes_sql, conn)

### Create labels

In [4]:
labels = match_data.apply(get_match_label, axis=1)
labels = labels.drop("match_api_id", axis=1)

match_data["label"] = labels

### Clean up some data and create new features

In [5]:
# make season numerical
match_data['season'] = match_data["season"].apply(lambda x: int(x.split("/")[0]))

# create new 'month' field
match_data['month'] = match_data["date"].apply(lambda x: int(x.split("-")[1]))

# TODO create fields which represent win/loss/draw percentage for each team (over the last x games/years)


### Remove unnecessary data

In [6]:
# shouldn't know these columns 
# TODO there are probably many more useless columns
columns_to_drop = ["match_api_id", "home_team_goal", "away_team_goal", "goal",
                   "shoton", "shotoff", "foulcommit", "card", "cross", "corner",
                   "possession", "date"]
match_data = match_data.drop(columns_to_drop, axis=1)


# drop home_player_X/Y and away_player_X/Y columns, not sure what they represent
for no in range(1,12):
    for char in ["X", "Y"]:
        match_data = match_data.drop("home_player_"+char+str(no), axis=1)
        match_data = match_data.drop("away_player_"+char+str(no), axis=1)
        
# TODO should probably drop rows with (a lot of) null values also (make sure to drop matching values from labels in this case)


### Join tables

In [7]:
# drop null values and duplicates from team_attributes, otherwise the dataset size will grow to 700k rows
team_attributes = team_attributes.dropna(subset=['team_api_id'])
team_attributes = team_attributes.drop_duplicates(subset=['team_api_id'])

match_data = match_data.dropna(subset=['home_team_api_id', 'away_team_api_id'])

# create 2 versions with home/away prefixes for joining purposes
team_attributes_home = team_attributes.add_prefix("home_")
team_attributes_away = team_attributes.add_prefix("away_")

# join match data with home and away team attributes

match_data = pd.merge(match_data, team_attributes_home, how="left", left_on="home_team_api_id", right_on="home_team_api_id")
match_data = pd.merge(match_data, team_attributes_away, how="left", left_on="away_team_api_id", right_on="away_team_api_id")



# join match data with home and away players' attributes
player_attributes = player_attributes.drop_duplicates(subset=['player_api_id'])
for i in range(1,12):
    
    home_player_attributes_i = player_attributes.add_prefix("home_p_" + str(i) + "_")
    away_player_attributes_i = player_attributes.add_prefix("away_p_" + str(i) + "_")
    
    match_data = pd.merge(match_data, home_player_attributes_i, left_on="home_player_" + str(i), right_on="home_p_" + str(i) + "_player_api_id")
    match_data = pd.merge(match_data, away_player_attributes_i, left_on="away_player_" + str(i), right_on="away_p_" + str(i) + "_player_api_id")
    
    # delete temp dataframe to conserve memory
    del(home_player_attributes_i)


In [8]:
match_data.columns.tolist()

['id',
 'country_id',
 'league_id',
 'season',
 'stage',
 'home_team_api_id',
 'away_team_api_id',
 'home_player_1',
 'home_player_2',
 'home_player_3',
 'home_player_4',
 'home_player_5',
 'home_player_6',
 'home_player_7',
 'home_player_8',
 'home_player_9',
 'home_player_10',
 'home_player_11',
 'away_player_1',
 'away_player_2',
 'away_player_3',
 'away_player_4',
 'away_player_5',
 'away_player_6',
 'away_player_7',
 'away_player_8',
 'away_player_9',
 'away_player_10',
 'away_player_11',
 'B365H',
 'B365D',
 'B365A',
 'BWH',
 'BWD',
 'BWA',
 'IWH',
 'IWD',
 'IWA',
 'LBH',
 'LBD',
 'LBA',
 'PSH',
 'PSD',
 'PSA',
 'WHH',
 'WHD',
 'WHA',
 'SJH',
 'SJD',
 'SJA',
 'VCH',
 'VCD',
 'VCA',
 'GBH',
 'GBD',
 'GBA',
 'BSH',
 'BSD',
 'BSA',
 'label',
 'month',
 'home_id',
 'home_team_fifa_api_id',
 'home_date',
 'home_buildUpPlaySpeed',
 'home_buildUpPlaySpeedClass',
 'home_buildUpPlayDribbling',
 'home_buildUpPlayDribblingClass',
 'home_buildUpPlayPassing',
 'home_buildUpPlayPassingClass',


### Select the columns to be used as features

In [9]:

features = match_data[['season',
                        'month',
                        'stage', 
                       
                        #'home_player_1',
                        #'home_player_2',
                        #'home_player_3',
                        #'home_player_4',
                        #'home_player_5',
                        #'home_player_6',
                        #'home_player_7',
                        #'home_player_8',
                        #'home_player_9',
                        #'home_player_10',
                        #'home_player_11',
                        #'away_player_1',
                        #'away_player_2',
                        #'away_player_3',
                        #'away_player_4',
                        #'away_player_5',
                        #'away_player_6',
                        #'away_player_7',
                        #'away_player_8',
                        #'away_player_9',
                        #'away_player_10',
                        #'away_player_11',
                       
                        'home_p_1_overall_rating',
                        'away_p_1_overall_rating',
                        'home_p_2_overall_rating',
                        'away_p_2_overall_rating',
                        'home_p_3_overall_rating',
                        'away_p_3_overall_rating',
                        'home_p_4_overall_rating',
                        'away_p_4_overall_rating',
                        'home_p_5_overall_rating',
                        'away_p_5_overall_rating',
                        'home_p_6_overall_rating',
                        'away_p_6_overall_rating',
                        'home_p_7_overall_rating',
                        'away_p_7_overall_rating',
                        'home_p_8_overall_rating',
                        'away_p_8_overall_rating',
                        'home_p_9_overall_rating',
                        'away_p_9_overall_rating',
                        'home_p_10_overall_rating',
                        'away_p_10_overall_rating',
                        'home_p_11_overall_rating',
                        'away_p_11_overall_rating',
                       
                        'home_buildUpPlaySpeed',
                        #'home_buildUpPlayDribbling',
                        #'home_buildUpPlayPassing',
                        #'home_chanceCreationPassing',
                        #'home_chanceCreationCrossing',
                        #'home_chanceCreationShooting',
                        'home_defencePressure',
                        'home_defenceAggression',
                        #'home_defenceTeamWidth',
                       
                        'away_buildUpPlaySpeed',
                        #'away_buildUpPlayDribbling',
                        #'away_buildUpPlayPassing',
                        #'away_chanceCreationPassing',
                        #'away_chanceCreationCrossing',
                        #'away_chanceCreationShooting',
                        'away_defencePressure',
                        'away_defenceAggression',
                        #'away_defenceTeamWidth',
                       
                        'B365H',
                        'B365D',
                        'B365A',
                        'BWH',
                        'BWD',
                        'BWA',
                        'label']]

features.head()

,season,month,stage,home_p_1_overall_rating,away_p_1_overall_rating,home_p_2_overall_rating,away_p_2_overall_rating,home_p_3_overall_rating,away_p_3_overall_rating,home_p_4_overall_rating,...,away_buildUpPlaySpeed,away_defencePressure,away_defenceAggression,B365H,B365D,B365A,BWH,BWD,BWA,label
0,2008,3,26,68.0,69.0,65.0,73.0,73.0,72.0,65.0,...,50.0,70.0,50.0,4.5,3.6,1.75,4.70,3.30,1.70,2.0
1,2014,12,21,67.0,78.0,68.0,69.0,67.0,73.0,69.0,...,35.0,60.0,70.0,8.0,4.5,1.40,6.25,4.33,1.48,0.0
2,2011,12,17,66.0,75.0,66.0,72.0,67.0,72.0,64.0,...,50.0,70.0,50.0,5.5,4.2,1.55,5.50,3.70,1.60,2.0
3,2012,8,3,66.0,75.0,65.0,72.0,64.0,76.0,63.0,...,50.0,70.0,50.0,3.6,3.6,1.95,3.75,3.40,1.85,0.0
4,2014,2,25,64.0,78.0,69.0,75.0,72.0,73.0,72.0,...,35.0,60.0,70.0,3.1,3.5,2.20,3.30,3.40,2.10,2.0


### Write data to csv

In [10]:
# drop rows with nulls
features = features.dropna()
features.to_csv("datasets/data.csv")

In [11]:
features.shape

(19591, 38)